In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [ ]:
import jax
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ICARUS.dynamical_systems.second_order_system import SecondOrderSystem
from ICARUS.dynamical_systems.integrate import (
    BackwardEulerIntegrator,
    ForwardEulerIntegrator,
    RK4Integrator,
    RK45Integrator,
    CrankNicolsonIntegrator,
    GaussLegendreIntegrator,
    NewmarkIntegrator,
)

# FEA Example

In [ ]:
# Create two finite elements
E = 1  # 235e9
A = 1  # np.pi * (0.3**2) / 4
L = 1.0
I = 1  # 30e-2
rho = 1  # 5490

In [ ]:
num_elems = 6

l = L / num_elems
Cm = rho * A
Ck = E * I
# element mass and stiffness matrices
m = (
    np.array(
        [
            # [140,       0,               0,             70,         0,              0           ], # x1
            [156, 22 * l, 54, -13 * l],  # y1
            [22 * l, 4 * l**2, 13 * l, -3 * l**2],  # Theta1
            # [70,         0,              0,             140,        0,              0           ], # x2
            [54, 13 * l, 156, -22 * l],  # y2
            [-13 * l, -3 * l**2, -22 * l, 4 * l**2],  # Theta2
        ]
    )
    * rho
    * A
    * l
    / 420
)


k = np.array(
    [
        # [E*A/l,     0,               0,             -E*A/l,     0,              0           ], # x1
        [
            12 * E * I / L**3,
            6 * E * I / L**2,
            -12 * E * I / L**3,
            6 * E * I / L**2,
        ],  # y1
        [6 * E * I / L**2, 4 * E * I / L, -6 * E * I / L**2, 2 * E * I / L],  # Theta1
        # [-E*A/l,    0,               0,             E*A/l,      0,              0           ], # x2
        [
            -12 * E * I / L**3,
            -6 * E * I / L**2,
            12 * E * I / L**3,
            -6 * E * I / L**2,
        ],  # y2
        [6 * E * I / L**2, 2 * E * I / L, -6 * E * I / L**2, 4 * E * I / L],  # Theta2
    ]
)

p = 1 / num_elems / l
q = 0.0
f_load = np.array(
    [
        # 0*l/2,      # x1
        p * l / 2,  # y1
        p * l**2 / 12,  # Theta1
        # 0*l/2,      # x2
        p * l / 2,  # y2
        -p * l**2 / 12,  # Theta2
    ]
)
# construct global mass and stiffness matrices
dof_per_elem = 2
M = np.zeros((dof_per_elem * (num_elems + 1), dof_per_elem * (num_elems + 1)))
K = np.zeros((dof_per_elem * (num_elems + 1), dof_per_elem * (num_elems + 1)))
C = np.zeros((dof_per_elem * (num_elems + 1), dof_per_elem * (num_elems + 1)))
f_ext = np.zeros((dof_per_elem * (num_elems + 1)))

# for each element, change to global coordinates
for i in range(num_elems):
    M_temp = np.zeros((dof_per_elem * (num_elems + 1), dof_per_elem * (num_elems + 1)))
    K_temp = np.zeros((dof_per_elem * (num_elems + 1), dof_per_elem * (num_elems + 1)))
    F_temp = np.zeros((dof_per_elem * (num_elems + 1)))

    # start = i:dof_per_elem
    # end = (i+2)*dof_per_elem
    M_temp[
        i * dof_per_elem : (i + 2) * dof_per_elem,
        i * dof_per_elem : (i + 2) * dof_per_elem,
    ] = m
    K_temp[
        i * dof_per_elem : (i + 2) * dof_per_elem,
        i * dof_per_elem : (i + 2) * dof_per_elem,
    ] = k
    F_temp[i * dof_per_elem : (i + 2) * dof_per_elem] = f_load
    M += M_temp
    K += K_temp
    f_ext += F_temp

restrained_dofs = [
    1,
    0,
]
# remove the fixed degrees of freedom
for dof in restrained_dofs:
    for i in [0, 1]:
        M = np.delete(M, dof, axis=i)
        K = np.delete(K, dof, axis=i)
        C = np.delete(C, dof, axis=i)
    f_ext = np.delete(f_ext, dof)

system = SecondOrderSystem(M, C, K, f_ext)
# test_all_integrators(system, jnp.zeros((2*M.shape[0])), 0.0, 100.0, 1e-4, compare_with_scipy = False)

In [ ]:
from scipy.linalg import eigh

evals, evecs = eigh(K, M)
freq = np.sqrt(evals)
print(evals)
print(freq[0])

In [ ]:
# # Newmark integrator
dt0 = 0.0001
newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)
x0 = jnp.zeros((2 * M.shape[0]))
t0 = 0.0
t_end = 30.0
t_data, x_data = newmark.simulate(x0, t0, t_end)

In [ ]:
import matplotlib.pyplot as plt

clip = 1000

fig, ax = plt.subplots(2, 1, figsize=(10, 10))

# Clip the data to only sample 1000 points
clip = jnp.maximum(1, int(len(t_data) / 1000))

ax[0].set_title("Displacement of node 1")
ax[0].plot(t_data[::clip], x_data[:, 0][::clip], label="x1")
ax[0].plot(t_data[::clip], x_data[:, 1][::clip], label="y1")
ax[0].plot(t_data[::clip], x_data[:, 2][::clip], label="theta1")
ax[0].legend()

ax[1].set_title("Displacement of node 2")
ax[1].plot(t_data[::clip], x_data[:, 3][::clip], label="x2")
ax[1].plot(t_data[::clip], x_data[:, 4][::clip], label="y2")
ax[1].plot(t_data[::clip], x_data[:, 5][::clip], label="theta2")
ax[1].legend()